In [33]:
using ODBC
using DataFrames
using DSWB
using Formatting

dsn = "dswb-natgeo" # Redshift esetTable(tableRt, tableType = "RESOURCE_TABLE")ndpoint
table = "beacons_4744" # beacon table name
tableRt = "beacons_4744_rt"

# Connect to Beacon Data
setRedshiftEndpoint(dsn)
setTable(tableRt, tableType = "RESOURCE_TABLE")
setTable(table)

Set Redshift endpoint done. 2017-04-21T15:01:54


"beacons_4744"

In [34]:
# Packages
include("/data/notebook_home/Production/Lib/Include-Package-v1.0.jl")

# Time values (tv.) structure created in include above, so init time here
#timeVariables(2017,1,2,2,30,2017,1,2,2,45);
#timeVariables(2017,3,20,10,0,2017,3,20,13,59);
weeklyTimeVariables(days=1);

April 19, 2017 23:59:00 to April 20, 2017 23:59:00 Local Time
April 20, 2017 03:59:00 to April 21, 2017 03:59:00 UTC Time


In [35]:
customer = "Nat Geo" 
#productPageGroup = "%"
productPageGroup = "%" # primary page group
localUrl = "%"
resourceUrl = "%ng-black-logo%"

localTable = "$(table)_Find_Any_Resource_view"
linesOutput = 250
minimumEncoded = 100000
;

In [36]:
try
    query("""\
        create or replace view $localTable as (
            select * from $table 
                where 
                    "timestamp" between $(tv.startTimeMsUTC) and $(tv.endTimeMsUTC) and 
                    page_group ilike '$(productPageGroup)' and
                    params_u ilike '$(localUrl)' 
        )
    """)
    cnt = query("""SELECT count(*) FROM $localTable""")
    println("$localTable count is ",cnt[1,1])
catch y
    println("setupLocalTable Exception ",y)
end

beacons_4744_Find_Any_Resource_view count is 2308778


In [37]:
function resourceSummary(localTable::ASCIIString,tableRt::ASCIIString;linesOut::Int64=25,minEncoded::Int64=1000)
    
    try
        joinTables = query("""\
        select 
        $tableRt.url, $tableRt.params_u, $localTable.url,
            count(*)
        from $localTable join $tableRt
        on $localTable.session_id = $tableRt.session_id and $localTable."timestamp" = $tableRt."timestamp"
        where 
        $tableRt.url ilike '$resourceUrl'
        group by 
        $tableRt.url, $tableRt.params_u, $localTable.url
        """);

        displayTitle(chart_title = "Resource Url Pattern $(resourceUrl)", chart_info = [tv.timeString], showTimeStamp=false)
        #scrubUrlToPrint(joinTables,limit=150)
        beautifyDF(joinTables[1:min(linesOut,end),:])
    catch y
        println("bigTable5 Exception ",y)
    end
end 


resourceSummary (generic function with 1 method)

In [38]:
resourceSummary(localTable,tableRt;linesOut=linesOutput,minEncoded=minimumEncoded)
;
#resourceSizes(localTable,tableRt,"%css";linesOut=linesOutput,minEncoded=minimumEncoded)

Resource Url Pattern %ng-black-logo% April 19, 2017 23:59:00 to April 20, 2017 23:59:00 Local Time

url,params u,url 1,count
http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/photography/photo-of-the-day/2017/04/operating-room-table/,http://www.nationalgeographic.com/,"5,897"
http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/travel/road-trips/flower-route-netherlands-road-trip/,http://www.nationalgeographic.com/,114
http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/sharks-underwater-volcano-sharkcano-kavachi/?utm_source=Facebook&utm_medium=Social&utm_content=link_fb20170419news-sharkano&utm_campaign&sf71590152=1,http://news.nationalgeographic.com/,"7,733"
http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/man-eating-lions-teeth-kenya/?utm_source=Facebook&utm_medium=Social&utm_content=link_fb20170419news-lionteeth&utm_campaign=Content&sf71526859=1,http://news.nationalgeographic.com/,"2,891"
http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/science/space/solar-system/mercury/,http://www.nationalgeographic.com/,141
http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/travel/photography/world-most-romantic-destinations/?utm_source=Facebook&utm_medium=Social&utm_content=link_fb20170325travel-romantic2&utm_campaign=Content&sf65693995=1,http://www.nationalgeographic.com/,10
http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/03/genius-traits-quiz-science/?utm_source=Twitter&utm_medium=Social&utm_campaign=Content&utm_content=link_twgn20170419traitsquiz-genius&sf71696127=1,http://news.nationalgeographic.com/,"1,292"
http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded567.png,http://www.nationalgeographic.com/magazine/2002/04/afghan-girl-revealed/,http://www.nationalgeographic.com/,173
http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/03/great-white-shark-numbers-rising-cape-cod/,http://news.nationalgeographic.com/,161
http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/animals/mammals/a/african-lion/,http://www.nationalgeographic.com/,175


In [39]:
function resourceSummary2(localTable::ASCIIString,tableRt::ASCIIString;linesOut::Int64=25,minEncoded::Int64=1000)
    
    try
        joinTables = query("""\
        select 
        url,params_u,count(*)
        from $tableRt
        where 
          url ilike '$resourceUrl' and
          "timestamp" between $(tv.startTimeMsUTC) and $(tv.endTimeMsUTC)
        group by 
            url, params_u
        order by count(*) desc
        """);

        displayTitle(chart_title = "Raw Resource Url Pattern $(resourceUrl)", chart_info = [tv.timeString], showTimeStamp=false)
        #scrubUrlToPrint(joinTables,limit=150)
        beautifyDF(joinTables[1:min(linesOut,end),:])
    catch y
        println("bigTable5 Exception ",y)
    end
end 


resourceSummary2 (generic function with 1 method)

In [40]:
resourceSummary2(localTable,tableRt;linesOut=linesOutput,minEncoded=minimumEncoded)
;
#resourceSizes(localTable,tableRt,"%css";linesOut=linesOutput,minEncoded=minimumEncoded)

Raw Resource Url Pattern %ng-black-logo% April 19, 2017 23:59:00 to April 20, 2017 23:59:00 Local Time

url,params u,count
http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/sharks-underwater-volcano-sharkcano-kavachi/?utm_source=Facebook&utm_medium=Social&utm_content=link_fb20170419news-sharkano&utm_campaign&sf71590152=1,"7,784"
http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/photography/photo-of-the-day/2017/04/operating-room-table/,"5,973"
http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/science-march-einstein-fbi-genius-science/?utm_source=Facebook&utm_medium=Social&utm_content=link_fb20170420news-einsteinactivist&utm_campaign&sf71587058=1,"4,740"
http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/ancient-egypt-mummies-statues-luxor-discovery/?utm_source=Facebook&utm_medium=Social&utm_content=link_fbp20170420news-egypttomb&utm_campaign=Content&sf71833340=1,"3,970"
http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/inca-khipus-code-discovery-peru/,"3,917"
http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/man-eating-lions-teeth-kenya/?utm_source=Facebook&utm_medium=Social&utm_content=link_fb20170419news-lionteeth&utm_campaign=Content&sf71526859=1,"2,911"
http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/ancient-egypt-mummies-statues-luxor-discovery/,"2,798"
http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/adventure/extreme-photo-of-the-week/surfing-in-teahupoo--tahiti-/,"2,452"
http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/adventure/trips/best-trails/worlds-best-hikes-dream-trails/,"2,201"
http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/london-national-park-greenspace-urban-conservation/?google_editors_picks=true,"2,178"


In [41]:
function resourceSummary3(localTable::ASCIIString,tableRt::ASCIIString;linesOut::Int64=25,minEncoded::Int64=1000)
    
    try
        joinTables = query("""\
        select 
        *
        from $tableRt
        where 
          url ilike '$resourceUrl' and
          "timestamp" between $(tv.startTimeMsUTC) and $(tv.endTimeMsUTC)
        limit 3
        """);

        displayTitle(chart_title = "Raw Resource Url Pattern $(resourceUrl)", chart_info = [tv.timeString], showTimeStamp=false)
        #scrubUrlToPrint(joinTables,limit=150)
        beautifyDF(joinTables[1:min(linesOut,end),:])
    catch y
        println("bigTable5 Exception ",y)
    end
end 


resourceSummary3 (generic function with 1 method)

In [42]:
resourceSummary3(localTable,tableRt;linesOut=linesOutput,minEncoded=minimumEncoded)
;
#resourceSizes(localTable,tableRt,"%css";linesOut=linesOutput,minEncoded=minimumEncoded)

Raw Resource Url Pattern %ng-black-logo% April 19, 2017 23:59:00 to April 20, 2017 23:59:00 Local Time

params u,session id,session start,timestamp,url,initiator type,start time,redirect start,redirect end,fetch start,dns start,dns end,tcp connection start,secure connection start,tcp connection end,request start,response first byte,response last byte,worker start,encoded size,transferred size,decoded size,height,width,x,y
http://news.nationalgeographic.com/2017/04/super-earth-planet-found-life-space-science/?utm_source=Facebook&utm_medium=Social&utm_content=link_fbp20170419news-superearth&utm_campaign&sf71568628=1,958588c8-7d2d-4618-8087-d58c60438b19-ooowk7,"1,492,660,951,126",2017-04-20T04:03:15.969,http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,img,651,0,0,651,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,NA,NA,NA
http://news.nationalgeographic.com/2017/04/sharks-underwater-volcano-sharkcano-kavachi/?utm_source=Facebook&utm_medium=Social&utm_content=link_fb20170419news-sharkano&utm_campaign&sf71590152=1,083b1952-9265-4548-a533-2aac504e7a41-ooowke,"1,492,660,958,146",2017-04-20T04:03:40.296,http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,img,"1,908",0,0,"1,908","1,913","1,913","1,913",0,"1,921","1,921","1,951","1,957",NA,NA,NA,NA,NA,NA,NA,NA
http://www.nationalgeographic.com/science/space/solar-system/the-sun/,043b3c83-0b6a-461f-829c-1ff82f159ca7-ooox7b,"1,492,661,783,188",2017-04-20T04:16:36.404,http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,img,396,0,0,396,397,399,399,0,399,464,489,491,NA,"4,601","5,006",NA,NA,NA,NA,NA


In [43]:
function resourceSummary4(localTable::ASCIIString,tableRt::ASCIIString;linesOut::Int64=25,minEncoded::Int64=1000)
    
    try
        joinTables = query("""\
        select 
        count(*),
        avg(start_time) as "Start Time",
        avg(fetch_start) as "Fetch S",
        avg(dns_end-dns_start) as "DNS ms",
        avg(tcp_connection_end-tcp_connection_start) as "TCP ms",
        avg(request_start) as "Req S",
        avg(response_first_byte) as "Req FB",	
        avg(response_last_byte) as "Req LB",
        max(response_last_byte) as "Max Req LB",
        url, params_u,
        avg(redirect_start) as "Redirect S",
        avg(redirect_end) as "Redirect E",
        avg(secure_connection_start) as "Secure Conn S"
        from $tableRt
        where 
          url ilike '$resourceUrl' and
          "timestamp" between $(tv.startTimeMsUTC) and $(tv.endTimeMsUTC)
        group by url, params_u
        order by count(*) desc
        """);

        displayTitle(chart_title = "Raw Resource Url Pattern $(resourceUrl)", chart_info = [tv.timeString], showTimeStamp=false)
        #scrubUrlToPrint(joinTables,limit=150)
        beautifyDF(joinTables[1:min(linesOut,end),:])
    catch y
        println("bigTable5 Exception ",y)
    end
end 


resourceSummary4 (generic function with 1 method)

In [44]:
resourceSummary4(localTable,tableRt;linesOut=linesOutput,minEncoded=minimumEncoded)
;
#resourceSizes(localTable,tableRt,"%css";linesOut=linesOutput,minEncoded=minimumEncoded)

Raw Resource Url Pattern %ng-black-logo% April 19, 2017 23:59:00 to April 20, 2017 23:59:00 Local Time

count,start time,fetch s,dns ms,tcp ms,req s,req fb,req lb,max req lb,url,params u,redirect s,redirect e,secure conn s
"7,784","1,614","1,614",38,13,"1,975","2,107","2,286","365,985",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/sharks-underwater-volcano-sharkcano-kavachi/?utm_source=Facebook&utm_medium=Social&utm_content=link_fb20170419news-sharkano&utm_campaign&sf71590152=1,0,0,0
"5,973","1,812","1,812",42,2,"1,912","2,235","2,291","161,009",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/photography/photo-of-the-day/2017/04/operating-room-table/,0,1,0
"4,740","1,918","1,918",61,20,"2,610","2,786","2,826","333,281",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/science-march-einstein-fbi-genius-science/?utm_source=Facebook&utm_medium=Social&utm_content=link_fb20170420news-einsteinactivist&utm_campaign&sf71587058=1,0,0,0
"3,970","1,672","1,672",32,11,"2,090","2,232","2,298","162,280",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/ancient-egypt-mummies-statues-luxor-discovery/?utm_source=Facebook&utm_medium=Social&utm_content=link_fbp20170420news-egypttomb&utm_campaign=Content&sf71833340=1,0,0,0
"3,917",773,773,28,2,"1,111","1,199","1,228","120,470",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/inca-khipus-code-discovery-peru/,0,0,0
"2,911","1,719","1,719",30,13,"2,235","2,383","2,762","921,544",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/man-eating-lions-teeth-kenya/?utm_source=Facebook&utm_medium=Social&utm_content=link_fb20170419news-lionteeth&utm_campaign=Content&sf71526859=1,0,0,0
"2,798",938,938,8,6,"1,174","1,299","1,322","151,296",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/ancient-egypt-mummies-statues-luxor-discovery/,0,1,0
"2,452","1,698","1,698",29,0,"1,866","2,009","2,049","243,240",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/adventure/extreme-photo-of-the-week/surfing-in-teahupoo--tahiti-/,0,0,0
"2,201",961,961,10,0,874,994,"1,167","289,240",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/adventure/trips/best-trails/worlds-best-hikes-dream-trails/,0,0,0
"2,178",771,771,15,12,"1,048","1,143","1,156","58,386",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/london-national-park-greenspace-urban-conservation/?google_editors_picks=true,0,0,0


In [45]:
function resourceSummary5(localTable::ASCIIString,tableRt::ASCIIString;linesOut::Int64=25,minEncoded::Int64=1000)
    
    try
        joinTables = query("""\
        select 
        (response_last_byte-start_time) as "Time Taken",
        (start_time) as "Start Time",
        (fetch_start) as "Fetch S",
        (dns_end-dns_start) as "DNS ms",
        (tcp_connection_end-tcp_connection_start) as "TCP ms",
        (request_start) as "Req S",
        (response_first_byte) as "Req FB",	
        (response_last_byte) as "Req LB",
        url, params_u,
        (redirect_start) as "Redirect S",
        (redirect_end) as "Redirect E",
        (secure_connection_start) as "Secure Conn S"
        from $tableRt
        where 
          url ilike '$resourceUrl' and
        "timestamp" between $(tv.startTimeMsUTC) and $(tv.endTimeMsUTC) and
        (response_last_byte-start_time) > 5000
        order by "Time Taken" desc
        """);

        displayTitle(chart_title = "Raw Resource Url Pattern $(resourceUrl)", chart_info = [tv.timeString], showTimeStamp=false)
        #scrubUrlToPrint(joinTables,limit=150)
        beautifyDF(joinTables[1:min(linesOut,end),:])
    catch y
        println("bigTable5 Exception ",y)
    end
end 


resourceSummary5 (generic function with 1 method)

In [46]:
resourceSummary5(localTable,tableRt;linesOut=linesOutput,minEncoded=minimumEncoded)
;
#resourceSizes(localTable,tableRt,"%css";linesOut=linesOutput,minEncoded=minimumEncoded)

Raw Resource Url Pattern %ng-black-logo% April 19, 2017 23:59:00 to April 20, 2017 23:59:00 Local Time

time taken,start time,fetch s,dns ms,tcp ms,req s,req fb,req lb,url,params u,redirect s,redirect e,secure conn s
"9,062,939",631,631,0,0,0,0,"9,063,570",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/travel/features/how-to-foster-a-love-of-travel-in-your-kids/,0,0,0
"5,889,487",421,421,0,0,0,0,"5,889,908",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/man-eating-lions-teeth-kenya/?utm_source=Facebook&utm_medium=Social&utm_campaign=NEWS&utm_content=link_fbwd20170420NEWSmaneatinglionsteethkenya&sf71550801=1,0,0,0
"2,800,826","1,635","1,635",0,0,0,0,"2,802,461",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/shipworm-giant-oceans-rare-bacteria/?utm_source=Facebook&utm_medium=Social&utm_content=link_fb20170417news-shipworm&utm_campaign=Content&sf71046384=1,0,0,0
"2,487,903","4,081","4,081",0,0,0,0,"2,491,984",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/travel/destinations/europe/spain/spain-lesser-known-unesco-world-heritage-sites/?utm_source=Facebook&utm_medium=Social&utm_content=link_fbt20170411spainunesco4333&utm_campaign=Content&sf69609537=1,0,0,0
"1,821,768",298,298,0,0,0,0,"1,822,066",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/photography/photos/patterns-flora/,0,0,0
"1,234,085","2,217","2,217",0,0,0,0,"1,236,302",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/man-eating-lions-teeth-kenya/,0,0,0
"919,503","2,041","2,041",0,0,0,0,"921,544",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/man-eating-lions-teeth-kenya/?utm_source=Facebook&utm_medium=Social&utm_content=link_fb20170419news-lionteeth&utm_campaign=Content&sf71526859=1,0,0,0
"809,846","1,015","1,015",0,0,0,0,"810,861",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/03/siberia-batagiaka-crater-climate-change/,0,0,0
"808,056",959,959,0,0,0,0,"809,015",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/giant-pandas-china-video-reintroduction/,0,0,0
"608,820",585,585,0,0,0,0,"609,405",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2016/12/supervolcano-campi-flegrei-stirs-under-naples-italy/,0,0,0


In [47]:
function resourceSummary6(localTable::ASCIIString,tableRt::ASCIIString;linesOut::Int64=25,minEncoded::Int64=1000)
    
    try
        joinTables = query("""\
        select 
        (response_last_byte-start_time) as "Time Taken",
        (start_time) as "Start Time",
        (fetch_start) as "Fetch S",
        (dns_end-dns_start) as "DNS ms",
        (tcp_connection_end-tcp_connection_start) as "TCP ms",
        (request_start) as "Req S",
        (response_first_byte) as "Req FB",	
        (response_last_byte) as "Req LB",
        url, params_u,
        (redirect_start) as "Redirect S",
        (redirect_end) as "Redirect E",
        (secure_connection_start) as "Secure Conn S"
        from $tableRt
        where 
          url ilike '$resourceUrl' and
        "timestamp" between $(tv.startTimeMsUTC) and $(tv.endTimeMsUTC) and
        start_time > 10000
        order by start_time desc
        """);

        displayTitle(chart_title = "Raw Resource Url Pattern $(resourceUrl)", chart_info = [tv.timeString], showTimeStamp=false)
        #scrubUrlToPrint(joinTables,limit=150)
        beautifyDF(joinTables[1:min(linesOut,end),:])
    catch y
        println("bigTable5 Exception ",y)
    end
end 


resourceSummary6 (generic function with 1 method)

In [48]:
resourceSummary6(localTable,tableRt;linesOut=linesOutput,minEncoded=minimumEncoded)
;
#resourceSizes(localTable,tableRt,"%css";linesOut=linesOutput,minEncoded=minimumEncoded)

Raw Resource Url Pattern %ng-black-logo% April 19, 2017 23:59:00 to April 20, 2017 23:59:00 Local Time

time taken,start time,fetch s,dns ms,tcp ms,req s,req fb,req lb,url,params u,redirect s,redirect e,secure conn s
460,"391,168","391,168",0,105,"391,502","391,603","391,628",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/photography/proof/2015/06/17/one-girls-tunnel-life-under-the-surface-in-bucharest/,0,0,0
347,"352,258","352,258",0,0,0,"352,588","352,605",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/photography/photo-of-the-day/,0,0,0
"10,943","286,875","286,875",15,19,"296,399","297,817","297,818",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/travel/travel-interests/arts-and-culture/how-instagram-is-changing-travel/,0,0,0
8,"263,803","263,803",0,0,"263,808","263,810","263,811",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.2fc3d452.png,http://www.nationalgeographic.com/animals/reptiles/a/alligator-snapping-turtle/,0,0,0
196,"259,353","259,353",42,65,"259,465","259,535","259,549",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/travel/features/2017-best-in-sustainable-tourism/sustainable-slovenia/,0,0,0
835,"242,405","242,405",0,0,"243,218","243,239","243,240",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/adventure/extreme-photo-of-the-week/surfing-in-teahupoo--tahiti-/,0,0,0
"1,114","229,306","229,306",0,0,"229,313","229,903","230,420",http://news.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://news.nationalgeographic.com/2017/04/north-pond-hermit-maine-knight-stranger-woods-finkel/,0,0,0
"1,566","229,222","229,222",0,0,"229,286","230,772","230,788",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/adventure/everest/cory-richards-adrian-ballinger-climbers-mount-everest-unfiltered-expedition-2017/?utm_source=NatGeocom&utm_medium=Email&utm_content=adventure_20170420&utm_campaign=Content&utm_rd=1940086186,0,0,0
"6,645","217,310","217,310",0,0,"217,316","223,948","223,955",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/environment/natural-disasters/tsunami-safety-tips/,0,0,0
"118,460","197,205","197,205",0,0,"296,391","310,156","315,665",http://www.nationalgeographic.com/etc/designs/platform/v2/images/logos/ng-black-logo.ngsversion.45ded159.png,http://www.nationalgeographic.com/travel/travel-interests/arts-and-culture/how-instagram-is-changing-travel/,0,0,0
